In [1]:
import sys
sys.path.append('..')
from scoring_pipeline import ScoringPipeline
import json
import os
import torch

/fs/nexus-scratch/vla/micromamba/envs/MMQ_LLAVA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# compute scores across 

results_dir = '/fs/cfar-projects/low-bit-vision/llava/gptq/gqa'
scorer = ScoringPipeline()

gather = []
for results_file in os.listdir(results_dir):

    results_path = os.path.join(results_dir, results_file)

    with open(results_path, 'r') as f:
        results = json.load(f)
        
        # post-processing llava output
        answers = results['answers']
        for ans in answers:
            ans['answer'] = ans['answer'].split('ASSISTANT: ')[-1]


        # print(scorer.compute_scores(answers, "gqa"))
        score = scorer.compute_scores(answers, "gqa")['acc']
        
        record = dict(
            vision_bits = results['vision_bits'],
            language_bits = results['language_bits'],
            acc = score
        )

        gather.append(record)
        


In [15]:
gather

[{'vision_bits': 6, 'language_bits': 5, 'acc': 61.34},
 {'vision_bits': 8, 'language_bits': 4, 'acc': 60.79},
 {'vision_bits': 3, 'language_bits': 5, 'acc': 59.59},
 {'vision_bits': 5, 'language_bits': 6, 'acc': 61.19},
 {'vision_bits': 6, 'language_bits': 6, 'acc': 61.33},
 {'vision_bits': 2, 'language_bits': 4, 'acc': 35.78},
 {'vision_bits': 6, 'language_bits': 16, 'acc': 61.27},
 {'vision_bits': 16, 'language_bits': 3, 'acc': 55.82},
 {'vision_bits': 3, 'language_bits': 2, 'acc': 0.0},
 {'vision_bits': 4, 'language_bits': 5, 'acc': 60.81},
 {'vision_bits': 16, 'language_bits': 16, 'acc': 61.46},
 {'vision_bits': 8, 'language_bits': 8, 'acc': 61.47},
 {'vision_bits': 6, 'language_bits': 4, 'acc': 60.8},
 {'vision_bits': 5, 'language_bits': 2, 'acc': 0.0},
 {'vision_bits': 8, 'language_bits': 6, 'acc': 61.35},
 {'vision_bits': 3, 'language_bits': 3, 'acc': 54.71},
 {'vision_bits': 6, 'language_bits': 3, 'acc': 56.26},
 {'vision_bits': 6, 'language_bits': 8, 'acc': 61.29},
 {'vision_b

In [16]:
import pandas as pd

df_gqa = pd.DataFrame(gather)


In [17]:
df_gqa

,vision_bits,language_bits,acc
0,6,5,61.34
1,8,4,60.79
2,3,5,59.59
3,5,6,61.19
4,6,6,61.33
5,2,4,35.78
6,6,16,61.27
7,16,3,55.82
8,3,2,0.00
9,4,5,60.81


In [18]:
df_gqa.to_csv('/fs/cfar-projects/low-bit-vision/final_results/llava/gptq_gqa.csv', index=None)

In [ ]:
# gqa ans
ans_path = '/fs/cfar-projects/low-bit-vision/llava/full_precision/gqa_test_do_pad/results_v8_l4.json'

with open(ans_path, 'r') as f:
    results = json.load(f)


{'answers': [{'question_id': '201307251', 'answer': 'No', 'gt_answer': 'no'}, {'question_id': '201640614', 'answer': 'Lady', 'gt_answer': 'women'}, {'question_id': '202225914', 'answer': 'No', 'gt_answer': 'no'}, {'question_id': '2062325', 'answer': 'Yes', 'gt_answer': 'yes'}, {'question_id': '201303229', 'answer': 'Tall', 'gt_answer': 'short'}, {'question_id': '201902997', 'answer': 'Computer', 'gt_answer': 'keyboard'}, {'question_id': '20567512', 'answer': 'Beach', 'gt_answer': 'ocean'}, {'question_id': '20136592', 'answer': 'Red', 'gt_answer': 'red'}, {'question_id': '20602803', 'answer': 'Brown', 'gt_answer': 'brown'}, {'question_id': '201079951', 'answer': 'Curtain', 'gt_answer': 'drapes'}, {'question_id': '201079952', 'answer': 'Curtain', 'gt_answer': 'drapes'}, {'question_id': '20982537', 'answer': 'Man', 'gt_answer': 'woman'}, {'question_id': '201079958', 'answer': 'Yes', 'gt_answer': 'yes'}, {'question_id': '202218649', 'answer': 'Picture', 'gt_answer': 'picture'}, {'question_

In [13]:
results.keys()

dict_keys(['answers', 'vision_bits', 'language_bits'])

In [ ]:
for res in results:
    res['answer'] = res['answer'].split('ASSISTANT: ')[-1]

def compute_gqa_results(results, scorer, save_path=None):
    gqa_results = scorer.compute_scores(results, "gqa")
    print(gqa_results)
# if save_path:
#     with open(save_path, "w") as f:
#         json.dump(gqa_results, f)



{'agg_metrics': 61.46, 'acc': 61.46}


In [36]:
def compute_gqa_results(results, scorer, save_path=None):
    gqa_results = scorer.compute_scores(results, "gqa")
    print(gqa_results)
    # if save_path:
    #     with open(save_path, "w") as f:
    #         json.dump(gqa_results, f)

In [6]:
# vqav2 ans
ans_path = '/fs/cfar-projects/low-bit-vision/llava/full_precision/vqav2_test_do_pad/results_v8_l4.json'

with open(ans_path, 'r') as f:
    results = json.load(f)


In [7]:
results.keys()

dict_keys(['answers', 'vision_bits', 'language_bits'])

In [8]:
answers = results['answers']
for ans in answers:
    ans['answer'] = ans['answer'].split('ASSISTANT: ')[-1]


In [10]:
results['answers'][0]

{'question_id': 262148000, 'answer': 'Down'}

In [11]:
ann_root = '/fs/cfar-projects/low-bit-vision/datasets/vqav2/annotations'
q_root = '/fs/cfar-projects/low-bit-vision/datasets/vqav2/questions'

# results["answers"] = answers
results["annotations"] = os.path.join(ann_root, "v2_mscoco_val2014_annotations.json")
results["questions"] = os.path.join(q_root, "v2_OpenEnded_mscoco_val2014_questions.json")

In [12]:
results.keys()

dict_keys(['answers', 'vision_bits', 'language_bits', 'annotations', 'questions'])

In [13]:

def compute_vqa_results(results, scorer, save_path=None):
    vqa_results = scorer.compute_scores(results, "vqav2")
    print(vqa_results)
    if save_path:
        with open(save_path, "w") as f:
            json.dump(vqa_results, f)



scorer = ScoringPipeline()
compute_vqa_results(results, scorer)

Adding current path to python system paths
loading VQA annotations and questions into memory...
0:00:09.746300
creating index...
index created!
Loading and preparing results...     
DONE (t=0.33s)
creating index...
index created!
computing accuracy
Finshed Percent: [####################] 99% Done computing accuracy
{'agg_metrics': 75.69, 'other': 68.86, 'yes/no': 91.04, 'number': 57.4}


In [43]:
len(results)

214354